In [1]:
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import torch.optim as optim
import torch 
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import gc
import torch.utils.data as D
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
class Lstm(nn.Module):
    def __init__(self, act_function, vocab_size, output_dim, embedding_dim, hidden_dim, n_layers, cuda, batch_size, drop_prob=0.5):
        super(Lstm, self).__init__()
        self.device = torch.device("cuda" if cuda else "cpu")
        self.act_function = act_function
        self.cuda = cuda
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        # initiate layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers)#, batch_first=True
        self.lin1 = nn.Linear(hidden_dim, hidden_dim) 
        self.lin2 = nn.Linear(hidden_dim, output_dim)

    def init_hidden(self, batch_size):
        #if self.cuda:
        #    return (torch.randn(self.n_layers, batch_size, self.hidden_dim).cuda(), torch.randn(self.n_layers, batch_size, self.hidden_dim).cuda())
        #else:
        #    return (torch.randn(self.n_layers, batch_size, self.hidden_dim), torch.randn(self.n_layers, batch_size, self.hidden_dim))
        return (torch.randn(self.n_layers, self.embedding_dim, self.hidden_dim), torch.randn(self.n_layers, self.embedding_dim, self.hidden_dim))
        
    def forward(self, x, hidden):
        x = x.long() 
        x = self.embedding(x) 
        x, hidden = self.lstm(x, hidden)
        x = self.act_function(self.lin1(x[:,-1,:]))
        x = self.act_function(self.lin2(x))
        return x, hidden